# Operações Básicas RDD

#### [Baseado em "Introduction to Spark with Python, by Jose A. Dianes"](https://github.com/jadianes/spark-py-notebooks)

## Fazendo download do Dataset

Neste notebook, usaremos o conjunto de dados reduzido 10% do fornecido para a KDD Cup 1999, contendo quase meio milhão de interações de rede. O arquivo é fornecido como um arquivo *Gzip* que será baixado localmente.

In [1]:
from urllib.request import urlretrieve

f = urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

## Criando o RDD

Agora podemos usar esse arquivo para criar nosso RDD.

In [3]:
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

## A tranformação `filter`

Essa transformação pode ser aplicada aos RDDs para manter apenas os elementos que satisfazem uma determinada condição. Mais concretamente, uma função é avaliada em cada elemento no RDD original. O novo RDD resultante conterá apenas os elementos que fazem a função retornar `True`.

Por exemplo, imagine que queremos contar quantas interações `normal` que temos em nosso conjunto de dados. Podemos filtrar nosso RDD `raw_data` da seguinte maneira.

In [4]:
normal_raw_data = raw_data.filter(lambda x: 'normal.' in x)

Agora podemos contar quantos elementos temos no novo RDD.

In [5]:
from time import time
t0 = time()
normal_count = normal_raw_data.count()
tt = time() - t0
print("There are {} 'normal' interactions".format(normal_count))
print("Count completed in {} seconds".format(round(tt,3)))

There are 97278 'normal' interactions
Count completed in 1.7 seconds


Lembre-se do bloco de notas 1 que temos um total de 494021 em nosso conjunto de dados de 10%. Aqui podemos ver que 97278 contém a palavra `normal`.

Observe que medimos o tempo decorrido para contar os elementos no RDD. Fizemos isso porque queríamos apontar que computações reais (distribuídas) no Spark acontecem quando executamos *actions* e não *transformations*. Neste caso, `count` é a ação que executamos no RDD. Podemos aplicar quantas transformações quisermos em um nosso RDD e nenhuma computação ocorrerá até chamarmos a primeira ação que, nesse caso, leva alguns segundos para ser concluída.

## A transformação `map`

Usando a transformação `map` no Spark, podemos aplicar uma função a todos os elementos do nosso RDD. Os lambdas do Python são especialmente expressivos para isso.

Neste caso, queremos ler nosso arquivo de dados como um arquivo formatado em CSV. Podemos fazer isso aplicando uma função lambda a cada elemento no RDD da seguinte maneira.

In [6]:
from pprint import pprint
csv_data = raw_data.map(lambda x: x.split(","))
t0 = time()
head_rows = csv_data.take(5)
tt = time() - t0
print("Parse completed in {} seconds".format(round(tt,3)))
pprint(head_rows[0])

Parse completed in 0.053 seconds
['0',
 'tcp',
 'http',
 'SF',
 '181',
 '5450',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '8',
 '8',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '1.00',
 '0.00',
 '0.00',
 '9',
 '9',
 '1.00',
 '0.00',
 '0.11',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 'normal.']


Mais uma vez, toda a ação acontece quando chamamos a primeira *action* do Spark, neste caso a operação *take*. E se pegarmos muitos elementos em vez de apenas os primeiros?

In [7]:
t0 = time()
head_rows = csv_data.take(100000)
tt = time() - t0
print("Parse completed in {} seconds".format(round(tt,3)))

Parse completed in 1.508 seconds


Podemos ver que demora mais tempo. A função `map` é aplicada agora de maneira distribuída a muitos elementos no RDD, portanto, o maior tempo de execução.

### Usando `map` e funções predefinidas

Claro que podemos usar funções pré-definidas com o `map`. Imagine que queremos ter cada elemento no RDD como um par de valores-chave em que a chave é a tag (por exemplo, *normal*) e o valor é toda a lista de elementos que representa a linha no arquivo formatado em CSV. Nós poderíamos proceder da seguinte forma.

In [13]:
def parse_interaction(line):
    elems = line.split(",")
    tag = elems[41]
    return (tag, elems)

key_csv_data = raw_data.map(parse_interaction)
head_rows = key_csv_data.take(5)
pprint(head_rows[0])

('normal.',
 ['0',
  'tcp',
  'http',
  'SF',
  '181',
  '5450',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '9',
  '9',
  '1.00',
  '0.00',
  '0.11',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'])


Isso foi fácil, não foi?

Em nosso notebook sobre como trabalhar com pares de valores-chave, usaremos esse tipo de RDDs para fazer agregações de dados (por exemplo, contagem por chave).

## A ação `collect`

Até agora usamos as ações `count` e` take`. Outra ação básica que precisamos aprender é `collect`. Basicamente, ele irá colocar todos os elementos do RDD na memória para que possamos trabalhar com eles. Por esse motivo, ele deve ser usado com cuidado, especialmente quando se trabalha com RDDs grandes.

Um exemplo usando nossos dados brutos.  

In [14]:
t0 = time()
all_raw_data = raw_data.collect()
tt = time() - t0
print("Data collected in {} seconds".format(round(tt,3)))

Data collected in 12.975 seconds


Isso levou mais tempo como qualquer outra ação que usamos antes, é claro. Cada nó do trabalhador do Spark que possui um fragmento do RDD precisa ser coordenado para recuperar sua parte e, em seguida, agrupar todos os dados em uma única memória.

Como último exemplo que combina todos os anteriores, queremos coletar todas as interações `normal` como pares de chave-valor.

In [15]:
# get data from file
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

# parse into key-value pairs
key_csv_data = raw_data.map(parse_interaction)

# filter normal key interactions
normal_key_interactions = key_csv_data.filter(lambda x: x[0] == "normal.")

# collect all
t0 = time()
all_normal = normal_key_interactions.collect()
tt = time() - t0
normal_count = len(all_normal)

print("Data collected in {} seconds".format(round(tt,3)))
print("There are {} 'normal' interactions".format(normal_count))

Data collected in 11.298 seconds
There are 97278 'normal' interactions
